This notebook is for Kaggle challenge: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

In [0]:
# !wget wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8,gu;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1566296813&Signature=CIhtnAXBUZldh5cG7aDiIV9FTK92sdj5d%2BgMvKzF23KzpyxcV748%2FZETmDWbHYVIvMwfjVUyoCda2K7jv%2BkXnh4BE4132G0Wy5SQkTzZ9lAiRQ%2B2ald3mtrwJigWRjK6KcvTOw%2FRsmfIoj%2BTSyW79kZ8sTwJ5a20Lj3g9%2BcMtremWVfBXQhNNNBHVMtEkmoAUNL6CzUTAkpBW9BIgjddg2%2FERCJEkXZKGU20fBWI732SuHGqHV2zIe0A%2FKPeYB0efENJR0XNpRil0Us90fkXV0xsII9h64Ef1oPX%2BdEXMxHqiHK6EBYaPAiSlRhHsro4xj2DumBqOj2NAKtHF%2BrIKQ%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip" -O "jigsaw-toxic-comment-classification-challenge.zip" -c
# !rm -rf sample_data
# !unzip jigsaw-toxic-comment-classification-challenge.zip
# !rm -rf jigsaw-toxic-comment-classification-challenge.zip

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [66]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [67]:
lens = train_df.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

(394.0732213246768, 590.7202819048923, 5000)

In [68]:
# append a "none" category
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_df['none'] = 1-train_df[label_cols].max(axis = 1)
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [0]:
# replace NA
train_df['comment_text'].fillna("unknown", inplace = True)
test_df['comment_text'].fillna("unknown", inplace = True)

In [70]:
train_df['comment_text'].str.replace('\n', ' ', regex = True)
test_df['comment_text'].str.replace('\n', ' ', regex = True)
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [71]:
# integer encode the documents
vocab_size = 4000
encoded_docs = [one_hot(d, vocab_size) for d in train_df['comment_text']]
# print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[1797 1356 3821 ...    0    0    0]
 [2992 3015 1589 ...    0    0    0]
 [ 316 1874 3511 ...    0    0    0]
 ...
 [2452 3065 3259 ...    0    0    0]
 [   5 3232 3822 ...    0    0    0]
 [   5 1923  118 ...    0    0    0]]


In [72]:
label_cols = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'none']]
print (label_cols.shape)

(159571, 7)


In [73]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
# print(model.summary())
# fit the model
model.fit(padded_docs, label_cols, epochs = 6)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, label_cols)
print(f'Accuracy: {accuracy*100}')

W0817 12:59:43.243883 140056492459904 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 1/6
159571/159571 [==============================] - 115s 720us/step - loss: 0.5598 - acc: 0.9360
Epoch 2/6
159571/159571 [==============================] - 114s 715us/step - loss: 0.4576 - acc: 0.9505
Epoch 3/6
159571/159571 [==============================] - 114s 715us/step - loss: 0.3922 - acc: 0.9650
Epoch 4/6
159571/159571 [==============================] - 114s 715us/step - loss: 0.3538 - acc: 0.9747
Epoch 5/6
159571/159571 [==============================] - 114s 717us/step - loss: 0.3351 - acc: 0.9796
Epoch 6/6
159571/159571 [==============================] - 9s 58us/step
Accuracy: 98.69023820117691


In [0]:
# integer encode the documents
vocab_size = 4000
encoded_docs = [one_hot(d, vocab_size) for d in test_df['comment_text']]
# print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
preds = model.predict_proba(padded_docs)

In [75]:
print (preds.shape)

(153164, 7)


In [0]:
submission_df = pd.concat([test_df.drop(['comment_text'], axis = 1), pd.DataFrame(preds[:, 0:6], columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])], axis = 1)

In [77]:
pd.read_csv('sample_submission.csv').head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [0]:
submission_df.to_csv('predictions2.csv', index=False)

In [79]:
submission_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1.000000e+00,0.476773,0.999945,0.000005,0.998073,0.010965
1,0000247867823ef7,5.960464e-08,0.000000,0.000000,0.000000,0.000000,0.000000
2,00013b17ad220c46,4.768372e-07,0.000000,0.000000,0.000000,0.000000,0.000000
3,00017563c3f7919a,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
4,00017695ad8997eb,1.642108e-05,0.000000,0.000000,0.000000,0.000000,0.000000


In [80]:
!zip subm.zip predictions2.csv

  adding: predictions2.csv (deflated 65%)
